In [13]:
import boto3
import boto3
import json
import time
import zipfile
from io import BytesIO



iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
lambda_client = boto3.client('lambda')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_description = "Agent for providing HR assistance to manage vacation time"
agent_instruction = "You are an HR agent, helping employees understand HR policies and manage vacation time"
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "Actions for getting the number of available vacations days for an employee and confirm new time off"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [8]:
inference_profile = "us.amazon.nova-micro-v1:0" 
foundation_model = inference_profile[3:]
foundation_model

'amazon.nova-micro-v1:0'

In [9]:
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:*::foundation-model/{foundation_model}",
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        },
        {
            "Sid": "AmazonBedrockAgentBedrockGetInferenceProfile",
            "Effect": "Allow",
            "Action":  [
                "bedrock:GetInferenceProfile",
                "bedrock:ListInferenceProfiles",
                "bedrock:UseInferenceProfile"
            ],
            "Resource": [
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [10]:
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {
            "Service": "bedrock.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

time.sleep(10)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'bc3e8bcf-6f4e-4d2b-974e-249658c5476a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 15 Aug 2025 05:21:07 GMT',
   'x-amzn-requestid': 'bc3e8bcf-6f4e-4d2b-974e-249658c5476a',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [14]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=inference_profile,
    instruction=agent_instruction,
)
agent_id = response['agent']['agentId']
agent_id, response

('CDL0KSVUWE',
 {'ResponseMetadata': {'RequestId': '2314dc57-ef30-4708-918f-e0d236bef43a',
   'HTTPStatusCode': 202,
   'HTTPHeaders': {'date': 'Fri, 15 Aug 2025 05:22:57 GMT',
    'content-type': 'application/json',
    'content-length': '692',
    'connection': 'keep-alive',
    'x-amzn-requestid': '2314dc57-ef30-4708-918f-e0d236bef43a',
    'x-amz-apigw-id': 'PVOXwGploAMEeAA=',
    'x-amzn-trace-id': 'Root=1-689ec431-1a23ca9e33690bfe46b39d37'},
   'RetryAttempts': 0},
  'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:447924746755:agent/CDL0KSVUWE',
   'agentCollaboration': 'DISABLED',
   'agentId': 'CDL0KSVUWE',
   'agentName': 'hr-assistant-function-def',
   'agentResourceRoleArn': 'arn:aws:iam::447924746755:role/AmazonBedrockExecutionRoleForAgents_hr-assistant-function-def',
   'agentStatus': 'CREATING',
   'createdAt': datetime.datetime(2025, 8, 15, 5, 22, 57, 543567, tzinfo=tzutc()),
   'description': 'Agent for providing HR assistance to manage vacation time',
   'foundationMo